# EAA Winter School in Computational Acoustics 2018

## 1D Finite-Element tutorial

### Preamble

We begin by loading a number of standard modules such as `numpy` and `matplotlib`. We also adjust some default parameters of figures.

In [115]:
import numpy as np
from numpy import exp, sqrt
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.rcParams["animation.html"] = "html5"
import matplotlib.animation
mpl.rc('lines', linewidth=2)
mpl.rc('font', size=14)
mpl.rc('axes', linewidth=1.5, labelsize=14)
mpl.rc('legend', fontsize=14)
from scipy.linalg import eig
from scipy.sparse.linalg import eigs

### Definition of the problem

We consider the 1D propagation in a tube. It is governed by the Helmholtz equation:
$$
p''(x)
+ k^2 p(x)
= 0 \textrm{ in } \Omega.
$$
where $p(x)$ is the acoustic pressure and $\Omega=]-L;0[$ is the geometrical domain associated to the tube. At the boundary of the domain, several different conditions can be considered.

The weak form associated to this problem is 
$$ 
\forall q\in V,\quad \Big[
p'(x)q(x)
\Big]_{-L}^0 - \displaystyle{\int_{-L}^0} p'(x)q'(x)dx + k^2 \displaystyle{\int_{-L}^0} p(x)q(x)dx =0 
$$



### Computation of modes of the tube

In a first time, we will compute the modes of the tube with closed ends on both sides. In that case, as $p'(-L)=p'(0)=0$, the bracket term of the weak form is zero. We will compute, by the Finite-Element Method the modes of the cavity. We know that analytically, in this case, the values possible values are discrete and given by:
$$ k_n=\dfrac{n\pi}{L},\quad n\in\mathbb{N}$$. So as to simplify the analysis, we will consider that the length of the tube is $L=\pi$:

In [116]:
L=np.pi

### Discretisation by a single linear element

We will implement the FEM for a single linear element. On the whole domain (element), the pressure and the test functions are assumed to be linear:
$$ p(x)=p_1\Big(1-\dfrac{x}{L}\Big)+p_2\dfrac{x}{L},\quad q(x)=q_1\Big(1-\dfrac{x}{L}\Big)+q_2\dfrac{x}{L},$$
where $p_1$ and $p_2$ (resp. $q_1$ and $q_2$) are the approximation of the pressure (resp. test function) in $x=-L$ and $x=0$. The weak form is then:

$$ 
\forall q_1,q_2\in \mathbb{R},\quad 
\{q_1,\,q_2\} \Big(
[\boldsymbol{H}_e(\delta)]-k^2[\boldsymbol{Q}_e(\delta)]\Big)\begin{Bmatrix}p_1\\p_2
\end{Bmatrix}
=0 
$$
with:
$$ [\boldsymbol{H}_e(\delta)]=\dfrac{1}{\delta}\begin{bmatrix}1&-1\\-1&1\end{bmatrix},\quad [\boldsymbol{Q}_e(\delta)]=\dfrac{\delta}{6}\begin{bmatrix}2&1\\1&2\end{bmatrix}.$$


In [117]:
def H_e(delta):
    return (1/delta)*np.array([[1.,-1.],[-1.,1.]])
def Q_e(delta):
    return (delta/6.)*np.array([[2.,1.],[1.,2.]])

$k$ are associated to the eigenvalues of the generalised eigenvalue problem:
$$ [\boldsymbol{H}_e(\delta)]\begin{Bmatrix}p_1\\p_2
\end{Bmatrix}=k^2[\boldsymbol{Q}_e(\delta)]\begin{Bmatrix}p_1\\p_2
\end{Bmatrix}.$$

In [118]:
delta = L
k = np.sqrt(eig( H_e(delta), Q_e(delta),left=False,right=False))
print(k)

[0.        +0.j 1.10265779+0.j]


We then obtain two eigenvalues. The first one is zero, explain why. The second is an approximation of $1$ as $L=\pi$. What is the quality of the approximation ?

### Discretisation by several linear elements:

We will now consider $n$ linear elements. $\Omega$ is divided in $n$ subdomains of length $\delta=L/n$ on which a linear approximation is considered. The pressure at the $n+1$ nodes are respectively denoted by $p_1$ ... $p_n$ and the weak form is:
$$ 
\forall q_1,...,q_n\in \mathbb{R},\quad \displaystyle{\sum_{k=1}^n}
\{q_k,\,q_{k+1}\} \Big(
[\boldsymbol{H}_e(\delta)]-k^2[\boldsymbol{Q}_e(\delta)]\Big)\begin{Bmatrix}p_k\\p_{k+1}
\end{Bmatrix}
=0.$$


What are, formally, the global matrices $[\boldsymbol{H}]$ and $[\boldsymbol{Q}]$ so that the weak form is rewritten:
$$ 
\forall q_1,...,q_n\in \mathbb{R},\quad 
\{q_1,\,...,q_n\} \Big(
[\boldsymbol{H}]-k^2[\boldsymbol{Q}]\Big)\begin{Bmatrix}p_1\\\vdots \\p_n
\end{Bmatrix}
=0.$$

In [119]:
n =4
delta = L/n
H =np.zeros((n+1,n+1))
Q =np.zeros((n+1,n+1))

for ie in range(n): # loop on the elements
    dof = slice(ie, ie+2) # indices of the local dofs
    H[dof,dof] += H_e(delta)
    Q[dof,dof] += Q_e(delta)
k = np.sort(np.sqrt(eigs(H, 2, Q,which='SM',return_eigenvectors=False)))
print(k)    
    

[1.11926428e-08+0.j 1.02585908e+00+0.j]


How many elements do we need to have a precision of 1\% ?

### Discretisation by a single quadratic element

We will now consider quadratic elements and will consider the case of a single element of length $\delta=L$. On this element, the pressure is approximated by a second order function:
$$ p(x)=p_1\Big(\dfrac{(x-L/2)(x-L)}{L^2/2}\Big)+p_2\Big(-\dfrac{x(x-L)}{L^2/2}\Big)+p_3\Big(\dfrac{(x)(x-L/2)}{L^2/2}\Big), $$
where $p_1$, $p_2$ and $p_3$ are in that case the approximation of the pressure in $x=0$, $x=L/2$ and $x=L$. A similar interpolation is considered for the test-function.

Plot on a paper or in the notebook the three shape functions.

The weak form is:

$$ 
\forall q_1,q_2,q_3\in \mathbb{R},\quad 
\{q_1,\,q_2,q_3\} \Big(
[\boldsymbol{H}_e'(\delta)]-k^2[\boldsymbol{Q}_e'(\delta)]\Big)\begin{Bmatrix}p_1\\p_2\\p_3
\end{Bmatrix}
=0 
$$
with:
$$ [\boldsymbol{H}_e'(\delta)]=\dfrac{1}{3\delta}\begin{bmatrix}7&-8&1\\-8&16&-8\\1&-8&7\end{bmatrix},\quad [\boldsymbol{Q}_e'(\delta)]=\dfrac{\delta}{30}\begin{bmatrix}
4& 2& -1 \\2 &16& 2\\-1& 2 &4
\end{bmatrix}.$$
 

In [120]:
def H_e_quad(delta):
    return (1/(3*delta))*np.array([[7.,-8.,1],[-8.,16.,-8],[1.,-8.,7]])
def Q_e_quad(delta):
    return (delta/30.)*np.array([[4.,2.,-1],[2.,16.,2],[-1,2,4]])

In [121]:
delta = L
k = np.sqrt(eig( H_e_quad(delta), Q_e_quad(delta),left=False,right=False))
print(k)

[2.46561778e+00+0.j 1.10265779e+00+0.j 9.12275720e-09+0.j]


In [122]:
n =2
delta = L/n
H =np.zeros((2*n+1,2*n+1))
Q =np.zeros((2*n+1,2*n+1))

for ie in range(n): # loop on the elements
    dof = slice(2*ie, 2*ie+3) # indices of the local dofs
    H[dof,dof] += H_e_quad(delta)
    Q[dof,dof] += Q_e_quad(delta)
k = np.sort(np.sqrt(eigs(H, 2, Q,which='SM',return_eigenvectors=False)))
print(k)    
    

[4.84655651e-09+0.j 1.00375412e+00+0.j]


### Convergence analysis

We will now consider the convergence of the method which is the quality of the approximation as a function of the number of degrees of freedom. What is the error for linear and quadratic elements for a number of nodes equal to:
1. n=11
2. n=101
3. n=1001

In [123]:
def erreur_lin(n):
    delta = L/n
    H =np.zeros((n+1,n+1))
    Q =np.zeros((n+1,n+1))
    for ie in range(n): # loop on the elements
        dof = slice(ie, ie+2) # indices of the local dofs
        H[dof,dof] += H_e(delta)
        Q[dof,dof] += Q_e(delta)
    k = np.sort(np.sqrt(eigs(H, 2, Q,which='SM',return_eigenvectors=False)))    
    return(k[1]-1.0)
    
def erreur_quad(n):    
    delta = L/n
    H =np.zeros((2*n+1,2*n+1))
    Q =np.zeros((2*n+1,2*n+1))

    for ie in range(n): # loop on the elements
        dof = slice(2*ie, 2*ie+3) # indices of the local dofs
        H[dof,dof] += H_e_quad(delta)
        Q[dof,dof] += Q_e_quad(delta)
    k = np.sort(np.sqrt(eigs(H, 2, Q,which='SM',return_eigenvectors=False)))
    return(k[1]-1.0)
    
print(erreur_lin(10))
print(erreur_quad(5))
print(erreur_lin(20))
print(erreur_quad(10))
print(erreur_lin(40))
print(erreur_quad(20)) 

(0.004117250605596645+0j)
(0.00010605175748512607+0j)
(0.0010283984338197438+0j)
(6.729780225311899e-06+0j)
(0.0002570407277571185+0j)
(4.2223687901632445e-07+0j)


Based on this quantitative observation. What can we say the error when the number of degrees of freedom is multiplied by 2 for linear and quadratic elements

In [125]:
print(erreur_lin(10)/erreur_lin(20))
print(erreur_quad(5)/erreur_quad(10))
print(erreur_lin(20)/erreur_lin(40))
print(erreur_quad(10)/erreur_quad(20))

(4.003555888678157+0j)
(15.75857651220608+0j)
(4.000916285246023+0j)
(15.938396588410741+0j)


If you want to go further, you can plot the convergence curve (ie. the error on the )